In [1]:
# Import our dependencies
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()

In [4]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()

In [6]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\sumai\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat, 1)

C:\Users\sumai\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=55, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the checkpoint path
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

FIRST ATTEMPT

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 80


batch_size=32
epochs=50
save_period=5
steps_per_epoch= len(X_train) / batch_size

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 120)               5280      
                                                                 
 dense_13 (Dense)            (None, 80)                9680      
                                                                 
 dense_14 (Dense)            (None, 1)                 81        
                                                                 
Total params: 15,041
Trainable params: 15,041
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
checkpoint = ModelCheckpoint(filepath = checkpoint_path, monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq=int(save_period*steps_per_epoch))

In [31]:
# Train the model
nn1.fit(X_train_scaled, y_train,
          batch_size=batch_size,
          epochs=epochs, steps_per_epoch=steps_per_epoch,
          verbose=1,
          validation_data=(X_test_scaled, y_test),
          callbacks=[checkpoint])

Epoch 1/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5693 - accuracy: 0.7218 - val_loss: 0.5573 - val_accuracy: 0.7271
Epoch 2/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5572 - accuracy: 0.7270 - val_loss: 0.5542 - val_accuracy: 0.7266
Epoch 3/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5544 - accuracy: 0.7312 - val_loss: 0.5530 - val_accuracy: 0.7277
Epoch 4/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5511 - accuracy: 0.7332 - val_loss: 0.5559 - val_accuracy: 0.7270
Epoch 5/50
790/803 [============================>.] - ETA: 0s - loss: 0.5500 - accuracy: 0.7309
Epoch 5: loss improved from inf to 0.55001, saving model to checkpoints\weights.05.hdf5
803/803 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7309 - val_loss: 0.5514 - val_accuracy: 0.7298
Epoch 6/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7323 - val_loss: 0.554

803/803 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 0.7381 - val_loss: 0.5544 - val_accuracy: 0.7296
Epoch 48/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5377 - accuracy: 0.7386 - val_loss: 0.5588 - val_accuracy: 0.7275
Epoch 49/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 0.7380 - val_loss: 0.5558 - val_accuracy: 0.7284
Epoch 50/50
787/803 [============================>.] - ETA: 0s - loss: 0.5371 - accuracy: 0.7394
Epoch 50: loss improved from 0.53782 to 0.53776, saving model to checkpoints\weights.50.hdf5
803/803 [==============================] - 1s 1ms/step - loss: 0.5374 - accuracy: 0.7390 - val_loss: 0.5576 - val_accuracy: 0.7285


In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5576 - accuracy: 0.7285 - 222ms/epoch - 828us/step
Loss: 0.5576013326644897, Accuracy: 0.7285131216049194


SECOND ATTEMPT

In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features2 = len(X_train_scaled[0])
hidden_nodes_layer1_2 = 80
hidden_nodes_layer2_2 = 30
hidden_nodes_layer3_2 = 10

batch_size_2=32
epochs_2=50
save_period_2=5
steps_per_epoch_2= len(X_train) / batch_size_2

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1_2, input_dim=number_input_features2, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 80)                3520      
                                                                 
 dense_22 (Dense)            (None, 30)                2430      
                                                                 
 dense_23 (Dense)            (None, 10)                310       
                                                                 
 dense_24 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
checkpoint_2 = ModelCheckpoint(filepath = checkpoint_path, monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq=int(save_period_2*steps_per_epoch_2))

In [44]:
# Train the model
nn2.fit(X_train_scaled, y_train,
          batch_size=batch_size_2,
          epochs=epochs_2, steps_per_epoch=steps_per_epoch_2,
          verbose=1,
          validation_data=(X_test_scaled, y_test),
          callbacks=[checkpoint_2])

Epoch 1/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5766 - accuracy: 0.7165 - val_loss: 0.5535 - val_accuracy: 0.7252
Epoch 2/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5553 - accuracy: 0.7280 - val_loss: 0.5524 - val_accuracy: 0.7273
Epoch 3/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5516 - accuracy: 0.7296 - val_loss: 0.5495 - val_accuracy: 0.7301
Epoch 4/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7304 - val_loss: 0.5498 - val_accuracy: 0.7296
Epoch 5/50
792/803 [============================>.] - ETA: 0s - loss: 0.5497 - accuracy: 0.7315
Epoch 5: loss improved from inf to 0.54939, saving model to checkpoints\weights.05.hdf5
803/803 [==============================] - 1s 1ms/step - loss: 0.5494 - accuracy: 0.7319 - val_loss: 0.5483 - val_accuracy: 0.7310
Epoch 6/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7323 - val_loss: 0.551

803/803 [==============================] - 1s 1ms/step - loss: 0.5390 - accuracy: 0.7388 - val_loss: 0.5535 - val_accuracy: 0.7290
Epoch 48/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7386 - val_loss: 0.5550 - val_accuracy: 0.7272
Epoch 49/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7375 - val_loss: 0.5516 - val_accuracy: 0.7308
Epoch 50/50
757/803 [===========================>..] - ETA: 0s - loss: 0.5386 - accuracy: 0.7371
Epoch 50: loss improved from 0.53959 to 0.53833, saving model to checkpoints\weights.50.hdf5
803/803 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7371 - val_loss: 0.5561 - val_accuracy: 0.7293


In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5561 - accuracy: 0.7293 - 161ms/epoch - 600us/step
Loss: 0.5560974478721619, Accuracy: 0.7293294668197632


THIRD ATTEMPT

In [46]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features3 = len(X_train_scaled[0])
hidden_nodes_layer1_3 = 80
hidden_nodes_layer2_3 = 30

batch_size_3=32
epochs_3=50
save_period_3=5
steps_per_epoch_3= len(X_train) / batch_size_3

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1_3, input_dim=number_input_features3, activation="sigmoid")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="sigmoid"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 80)                3520      
                                                                 
 dense_26 (Dense)            (None, 30)                2430      
                                                                 
 dense_27 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [48]:
checkpoint_3 = ModelCheckpoint(filepath = checkpoint_path, monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq=int(save_period_3*steps_per_epoch_3))

In [49]:
# Train the model
nn3.fit(X_train_scaled, y_train,
          batch_size=batch_size_3,
          epochs=epochs_3, steps_per_epoch=steps_per_epoch_3,
          verbose=1,
          validation_data=(X_test_scaled, y_test),
          callbacks=[checkpoint_3])

Epoch 1/50
803/803 [==============================] - 2s 1ms/step - loss: 0.5955 - accuracy: 0.7036 - val_loss: 0.5768 - val_accuracy: 0.7192
Epoch 2/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5728 - accuracy: 0.7223 - val_loss: 0.5697 - val_accuracy: 0.7219
Epoch 3/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5681 - accuracy: 0.7244 - val_loss: 0.5672 - val_accuracy: 0.7214
Epoch 4/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5632 - accuracy: 0.7268 - val_loss: 0.5586 - val_accuracy: 0.7248
Epoch 5/50
786/803 [============================>.] - ETA: 0s - loss: 0.5594 - accuracy: 0.7284
Epoch 5: loss improved from inf to 0.55904, saving model to checkpoints\weights.05.hdf5
803/803 [==============================] - 1s 1ms/step - loss: 0.5591 - accuracy: 0.7284 - val_loss: 0.5563 - val_accuracy: 0.7258
Epoch 6/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5573 - accuracy: 0.7288 - val_loss: 0.556

803/803 [==============================] - 1s 1ms/step - loss: 0.5417 - accuracy: 0.7366 - val_loss: 0.5496 - val_accuracy: 0.7279
Epoch 48/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7375 - val_loss: 0.5496 - val_accuracy: 0.7296
Epoch 49/50
803/803 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7363 - val_loss: 0.5502 - val_accuracy: 0.7279
Epoch 50/50
769/803 [===========================>..] - ETA: 0s - loss: 0.5400 - accuracy: 0.7374
Epoch 50: loss improved from 0.54137 to 0.54067, saving model to checkpoints\weights.50.hdf5
803/803 [==============================] - 1s 1ms/step - loss: 0.5409 - accuracy: 0.7367 - val_loss: 0.5501 - val_accuracy: 0.7301


In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5501 - accuracy: 0.7301 - 234ms/epoch - 875us/step
Loss: 0.5500807166099548, Accuracy: 0.7301457524299622


In [51]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimization.h5")